## This python notebook has examples of the Max Coverage Problem
### load libraries

In [18]:
import geopandas as gpd
import pandas as pd
import os, sys, time
sys.path.append(r'/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

### read back in processed graph

In [19]:
G = nx.read_gpickle('./sampleData/nouakchott/biggest_subg.pickle')

In [20]:
gn.example_edge(G)

(0, 23857, {'Wkt': 'LINESTRING (-15.8975893 18.0394637, -15.8976921 18.039127)', 'id': 45978, 'infra_type': 'residential', 'osm_id': '667426151', 'key': 'edge_45978', 'length': 38.82328422750167, 'Type': 'legitimate', 'time': 13.976382321900601, 'mode': 'drive'})


## load origins and destinations

In [21]:
origins = pd.read_csv('./sampleData/nouakchott/origins_test1.csv')
origins['geometry'] = list(zip(origins['Lon'],origins['Lat']))
origins['geometry'] = origins['geometry'].apply(Point)
origins_gdf = gpd.GeoDataFrame(origins, crs = {'init':'epsg:4326'}, geometry = 'geometry')
origins_gdf = gn.pandana_snap(G, origins_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
origins = list(origins_gdf.NN)
origins = list(set(origins))

In [22]:
destinations = pd.read_csv('./sampleData/nouakchott/destinations_test1.csv')
destinations['geometry'] = list(zip(destinations['Lon'],destinations['Lat']))
destinations['geometry'] = destinations['geometry'].apply(Point)
destinations_gdf = gpd.GeoDataFrame(destinations, crs = {'init':'epsg:4326'}, geometry = 'geometry')
destinations_gdf = gn.pandana_snap(G, destinations_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
#destinations_gdf.NN is the nearest node of the road network
destinations = list(destinations_gdf.NN)
destinations = list(set(destinations))

## calculate OD Matrix

In [8]:
%time OD = gn.calculate_OD(G, origins, destinations, fail_value = 9999999999999)

CPU times: user 2.36 s, sys: 0 ns, total: 2.36 s
Wall time: 2.7 s


In [9]:
OD_df = pd.DataFrame(OD, columns = destinations, index = origins)

In [10]:
import importlib
importlib.reload(gn)

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.3 
osmnx version: 0.9 


<module 'GOSTnet' from '/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py'>

## Max Coverage Problem
### Objective: Determine the location of P facilities in order to maximize the demand covered within a pre-specified maximum distance coverage

### we need to produce a series that has each origin and its respective population

In [11]:
origins_w_demands_series = pd.Series(origins_gdf.demand.values,index=origins_gdf.NN)

### inputs include locating 3 facilities, a pre-specified maximum distance coverage of 2000 seconds, and a series of origins with their population

In [16]:
max_coverage_result = gn.optimize_max_coverage(OD_df, p_facilities = 3, max_coverage = 2000, origins_pop_series = origins_w_demands_series, existing_facilities = None)

print objective value
12637353.0


In [17]:
max_coverage_result

[15739, 1646, 4965]